In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np 
import math
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import Huber
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv("your dataset")

df.drop(columns=['Unnamed: 0', 'date', 'serial_number', 'model','capacity_bytes','datacenter','cluster_id', 'vault_id',
       'pod_id', 'pod_slot_num', 'is_legacy_format', ], errors='ignore', inplace=True)
threshold = len(df) * 0.8

df = df.dropna(axis=1,thresh=threshold)
df=df.dropna()
kcolumns=["Selected features"]
dfraw=df[kcolumns]
print(dfraw.columns)
dfraw.drop(columns=['failure'], errors='ignore', inplace=True)

X = dfraw.drop(columns=['label'])
y = dfraw['label'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.02,
    subsample=0.7,
    colsample_bytree=0.7,
    gamma=3,
    reg_lambda=8,
    alpha=5,
    min_child_weight=3,
    tree_method="hist",
    eval_metric="aucpr",
    random_state=42
)

rf_model = RandomForestClassifier(
    class_weight='balanced',
    n_estimators=200,
    max_depth=22,
    min_samples_split=6,
    min_samples_leaf=1,
    max_features='sqrt',
    bootstrap=True,
    random_state=42
)

lr_model = LogisticRegression()

voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_model), ('rf', rf_model)],
    voting='hard'
)

voting_clf.fit(X_train, y_train)

y_pred1 = voting_clf.predict(X_test)

fault_indices = y_test[y_pred1 == 1].index
X_fault = X_test.loc[fault_indices]
y_fault = y_test.loc[fault_indices]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

time_steps = 42  
n_features = X_scaled.shape[1]
features_per_step = n_features  

if n_features % features_per_step == 0:
    time_steps = min(time_steps, n_features // features_per_step)
    features_per_time_step = n_features // time_steps
else:

    features_per_time_step = n_features // time_steps
    if features_per_time_step * time_steps < n_features:
        features_per_time_step += 1
        padding_needed = features_per_time_step * time_steps - n_features
        if padding_needed > 0:
            X_scaled = np.pad(X_scaled, ((0, 0), (0, padding_needed)), 'constant')
            n_features = X_scaled.shape[1]
X_lstm = X_scaled.reshape((X_scaled.shape[0], time_steps, features_per_time_step))

print(f"重塑后的数据形状: {X_lstm.shape}")

X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

inputs = layers.Input(shape=(time_steps, features_per_time_step))

x = layers.Bidirectional(layers.LSTM(512, activation='tanh', return_sequences=True, recurrent_dropout=0.1))(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Bidirectional(layers.LSTM(256, activation='tanh', return_sequences=True, recurrent_dropout=0.1))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Bidirectional(layers.LSTM(128, activation='tanh', return_sequences=True, recurrent_dropout=0.1))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Bidirectional(layers.LSTM(64, activation='tanh', recurrent_dropout=0.1))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

def calculate_feature_frequencies(X):
    feature_occurrences = np.sum(X != 0, axis=0)
    total_samples = X.shape[0]
    frequencies = feature_occurrences / total_samples
    return frequencies

feature_frequencies = calculate_feature_frequencies(X)

class RareFeatureWeightedLoss(tf.keras.losses.Loss):
    def __init__(self, original_loss=tf.keras.losses.Huber(delta=1.0), lambda_reg=0.01, epsilon=1e-6, feature_frequencies=None, **kwargs):
        super().__init__(**kwargs)
        self.original_loss = original_loss
        self.lambda_reg = lambda_reg
        self.epsilon = epsilon
        self.feature_frequencies = feature_frequencies
        
        if feature_frequencies is not None:
            self.rare_weights = tf.cast(1.0 / (feature_frequencies + self.epsilon), tf.float32)
        else:
            self.rare_weights = None
    
    def call(self, y_true, y_pred):
        original_loss_value = self.original_loss(y_true, y_pred)
        if self.rare_weights is None:
            return original_loss_value
        regularization_term = tf.reduce_sum(tf.exp(-self.rare_weights))
        
        lambda_reg = tf.cast(self.lambda_reg, tf.float32)
        
        original_loss_value = tf.cast(original_loss_value, tf.float32)
        
        regularization_term = tf.cast(regularization_term, tf.float32)
        
        total_loss = original_loss_value + lambda_reg * regularization_term
        
        return total_loss

custom_loss = RareFeatureWeightedLoss(
    original_loss=tf.keras.losses.Huber(delta=1.0),
    lambda_reg=0.01,
    epsilon=1e-6,    
    feature_frequencies=feature_frequencies
)
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss=custom_loss, metrics=['mae'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-4, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=1)

history = model.fit(
    X_train_lstm, y_train_lstm,
    epochs=300,
    batch_size=64,
    validation_data=(X_test_lstm, y_test_lstm),
    callbacks=[lr_scheduler, early_stopping]
)

y_pred = model.predict(X_test_lstm)
loss, mae = model.evaluate(X_test_lstm, y_test_lstm)
rmse = np.sqrt(mean_squared_error(y_test_lstm, y_pred))

print(f"测试损失: {loss}")
print(f"测试MAE: {mae}")
print(f"RMSE: {rmse:.2f} 天")

y_true = np.array(y_test_lstm)

y_pred_flat = y_pred.flatten()

predicted_fault_indices = np.where(y_pred_flat <= 5)[0]

correct_indices = np.where(np.abs(y_pred_flat - y_true) <= 5)[0]

accuracy_5days = len(correct_indices) / len(y_true)

if len(predicted_fault_indices) > 0:
    precision_5days = len(set(predicted_fault_indices) & set(correct_indices)) / len(predicted_fault_indices)
else:
    precision_5days = 0

print(f"5天内预测正确的准确率: {accuracy_5days:.4f}")
print(f"5天内预测正确的精确率: {precision_5days:.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="val loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("model train loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(y_test_lstm, y_pred, alpha=0.5)
plt.plot([min(y_test_lstm), max(y_test_lstm)], [min(y_test_lstm), max(y_test_lstm)], 'r--')
plt.xlabel("true")
plt.ylabel("pre")
plt.title("model train pre")
plt.grid(True)
plt.tight_layout()
plt.show()

predicted_failure_times = model.predict(X_test_lstm)
predicted_failure_times = np.maximum(predicted_failure_times, 0.11)
predicted_failure_times = np.round(predicted_failure_times, 2)

if len(X_fault) > 0:
    print("\n===== 对预测为故障的样本进行分析 =====")
    
    fault_original_indices = []
    for idx in fault_indices:
        original_idx = X.index.get_loc(idx)
        fault_original_indices.append(original_idx)
    
    X_fault_lstm = []
    y_fault_actual = []
    
    for idx in fault_original_indices:
        fault_data = X_scaled[idx].reshape(1, -1)
        fault_data_reshaped = fault_data.reshape(1, time_steps, features_per_time_step)
        X_fault_lstm.append(fault_data_reshaped[0])
        y_fault_actual.append(y.iloc[idx])
    
    if len(X_fault_lstm) > 0:
        X_fault_lstm = np.array(X_fault_lstm)
        y_fault_actual = np.array(y_fault_actual)     
        print(f"故障样本数量: {len(X_fault_lstm)}")
        y_pred_fault = model.predict(X_fault_lstm)
        mae_fault = mean_absolute_error(y_fault_actual, y_pred_fault)
        rmse_fault = np.sqrt(mean_squared_error(y_fault_actual, y_pred_fault))
        
        print(f"故障样本MAE: {mae_fault:.4f}")
        print(f"故障样本RMSE: {rmse_fault:.2f} 天")
        
        plt.figure(figsize=(8, 6))
        plt.scatter(y_fault_actual, y_pred_fault, alpha=0.7, color='blue')
        plt.plot([min(y_fault_actual), max(y_fault_actual)], 
                 [min(y_fault_actual), max(y_fault_actual)], 'k--')
        plt.xlabel("true time")
        plt.ylabel("pre time")
        plt.title("sample pre")
        plt.grid(True)
        plt.show()